In [3]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Add, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from keras.preprocessing.image import ImageDataGenerator
from sklearn import preprocessing
import locale
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from deepforest import CascadeForestClassifier
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import ReLU, AvgPool2D
# from tensorflow.keras.layers import Input, DepthwiseConv2D
# from tensorflow.keras.layers import Conv2D, BatchNormalization
# from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
# from tensorflow.keras import Model

In [4]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/data5 (1).csv")
data_dir = '/Users/Eddie/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        # print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
print(images[0])
print(images.shape)

1000


[[[3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  [3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  [3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  ...
  [0.0000000e+00 2.1222609e-03 5.6901196e-04 ... 0.0000000e+00
   2.1991543e-03 9.2272203e-05]
  [0.0000000e+00 2.0299887e-03 7.6893502e-04 ... 0.0000000e+00
   1.9684739e-03 9.3810074e-04]
  [0.0000000e+00 1.9684739e-03 9.3810074e-04 ... 0.0000000e+00
   2.0607461e-03 6.1514805e-04]]

 [[3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  [3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  [3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  ...
  [0.0000000e+00 2.1222609e-03 5.3825456e-04 ... 0.0000000e+00
   2.1683970e-03 2.4605924e-04]
  [0.0000000e+00 2.0915035e-03 6.459

In [5]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)

(800, 2)
(200, 2)
(800,)
(200,)


In [6]:
def create_cnn_alexnet():
    model = Sequential()
    model.add(Conv2D(filters = 96, input_shape = (320, 320, 9),  
                kernel_size = (11, 11), strides = (4, 4),  
                padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling  
    model.add(MaxPooling2D(pool_size = (2, 2), 
                strides = (2, 2), padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 2nd Convolutional Layer 
    model.add(Conv2D(filters = 256, kernel_size = (11, 11),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling 
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 3rd Convolutional Layer 
    model.add(Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 4th Convolutional Layer 
    model.add(Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 5th Convolutional Layer 
    model.add(Conv2D(filters = 256, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling 
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # Flattening 
    model.add(Flatten()) 
    
    # 1st Dense Layer 
    model.add(Dense(4096, input_shape = (224*224*3, ))) 
    model.add(Activation('relu')) 
    # Add Dropout to prevent overfitting 
    model.add(Dropout(0.4)) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 2nd Dense Layer 
    model.add(Dense(4096)) 
    model.add(Activation('relu')) 
    # Add Dropout 
    model.add(Dropout(0.4)) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    model.add(Dense(2048, activation = 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation = 'relu'))
    return model

In [7]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [8]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1',)(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')         (X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [9]:
def ResNet50(input_shape=(320, 320, 9)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', )(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    headModel = Flatten()(X)
    headModel=Dense(256, activation='relu', name='fc1')(headModel)
    headModel=Dense(128, activation='relu', name='fc2')(headModel)
    headModel = Dense( 1, name='fc3')(headModel)
    model = Model(inputs=X_input, outputs=headModel, name='ResNet50')
    return model


In [10]:
def create_cnn_resnet():
    model = Sequential()
    model.add(Conv2D(64,input_shape = (320,320,9), kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu'))
    model.add(AveragePooling2D(pool_size = 2))
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size = 2, padding = 'same'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(2048, activation = 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(1,activation = 'relu'))

    return model

In [11]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def create_cnn_mobilenet():
    input = Input(shape = (320,320,9))
    x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # main part of the model
    x = mobilnet_block(x, filters = 64, strides = 1)
    x = mobilnet_block(x, filters = 128, strides = 2)
    x = mobilnet_block(x, filters = 128, strides = 1)
    x = mobilnet_block(x, filters = 256, strides = 2)
    x = mobilnet_block(x, filters = 256, strides = 1)
    x = mobilnet_block(x, filters = 512, strides = 2)
    for _ in range (5):
        x = mobilnet_block(x, filters = 512, strides = 1)
    x = mobilnet_block(x, filters = 1024, strides = 2)
    x = mobilnet_block(x, filters = 1024, strides = 1)
    x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
    x = Flatten()(x)
    x = Dense(units = 1024, activation = 'relu')(x)
    output = Dropout(0.4)(x)
    output = Dense (1)(output)
    model = Model(inputs=input, outputs=output)
    # model.summary()

    return model

In [12]:
alexnet = create_cnn_alexnet()
alexnet.compile(loss = None)
alexnetPredict = alexnet.predict(trainImagesX)
print(alexnetPredict.shape)




25/25 [==============================] - 15s 566ms/step
(800, 1)


In [13]:
resnet = ResNet50()
resnetPredict = resnet.predict(trainImagesX)
print(resnetPredict.shape)

25/25 [==============================] - 44s 2s/step
(800, 1)


In [14]:
mobilenet = create_cnn_mobilenet()
mobilenetPredict = mobilenet.predict(trainImagesX)
print(mobilenetPredict.shape)

25/25 [==============================] - 13s 488ms/step
(800, 1)


In [15]:
alexnetPredict = np.array(alexnetPredict)
resnetPredict = np.array(resnetPredict)
mobilenetPredict = np.array(mobilenetPredict)
trainAttrX = np.array(trainAttrX)
trainAttrX.resize(800,2)
trainData = concatenate([alexnetPredict,resnetPredict,mobilenetPredict,trainAttrX])
trainData = np.array(trainData)
trainData.reshape(-1,1)

print(trainData.shape)

(800, 5)


In [16]:
model = CascadeForestClassifier(n_estimators=50,random_state=1,backend='sklearn')
model.fit(trainData,trainy)

[2023-12-27 22:47:12.514] Start to fit the model:
[2023-12-27 22:47:12.514] Fitting cascade layer = 0 
[2023-12-27 22:47:29.335] layer = 0  | Val Acc = 99.375 % | Elapsed = 16.820 s
[2023-12-27 22:47:29.347] Fitting cascade layer = 1 
[2023-12-27 22:47:49.215] layer = 1  | Val Acc = 99.125 % | Elapsed = 19.868 s
[2023-12-27 22:47:49.215] Early stopping counter: 1 out of 2
[2023-12-27 22:47:49.226] Fitting cascade layer = 2 
[2023-12-27 22:48:05.980] layer = 2  | Val Acc = 99.000 % | Elapsed = 16.754 s
[2023-12-27 22:48:05.980] Early stopping counter: 2 out of 2
[2023-12-27 22:48:05.980] Handling early stopping
[2023-12-27 22:48:05.985] The optimal number of layers: 1


In [17]:
print(model.get_layer_feature_importances(0))

[0.00179433 0.17522548 0.09631472 0.2476071  0.47905836]


In [18]:
model.save('/Users/Eddie/Downloads/savetest')

RuntimeError: The directory to be created already exists /Users/Eddie/Downloads/savetest.

In [32]:
gasdata = pd.read_csv("/Users/eddie/Downloads/Gas Sensors Data - Sheet1 (3) (1).csv")
result = gasdata['Gas']
gasdata = gasdata.drop(columns=['Gas'])
gasdata= (gasdata - np.min(gasdata)) / (np.max(gasdata) - np.min(gasdata))
images =  sorted(os.listdir("/Users/eddie/Downloads/No Gas 4/No Gas"))
X_dat = []
for image in images:
       #print(image)
       img = mpimg.imread('/Users/eddie/Downloads/No Gas 4/No Gas/'+image)
       img.resize(320,320,9)
       img = img/255.0
       X_dat.append(img)
test_images = np.array(X_dat)
print("gay")
print(test_images[1])
print(test_images.shape)

gay
[[[0.0000000e+00 3.1372549e-03 3.2602844e-03 ... 3.2295270e-03
   3.9215689e-03 0.0000000e+00]
  [3.4755864e-03 3.1987696e-03 3.9215689e-03 ... 3.9215689e-03
   0.0000000e+00 3.2756634e-03]
  [3.2449057e-03 3.9215689e-03 0.0000000e+00 ... 0.0000000e+00
   3.0911188e-03 3.2756634e-03]
  ...
  [3.5986160e-03 3.1064975e-03 3.9215689e-03 ... 3.9215689e-03
   3.0757405e-05 3.5678586e-03]
  [3.1372549e-03 3.9215689e-03 1.5378702e-05 ... 1.5378702e-05
   3.5371012e-03 3.1680122e-03]
  [3.9215689e-03 4.6136101e-05 3.5524799e-03 ... 3.5832373e-03
   3.0603614e-03 3.9215689e-03]]

 [[1.0765091e-04 3.5986160e-03 3.0142253e-03 ... 2.9527105e-03
   3.9215689e-03 1.6916571e-04]
  [3.6293734e-03 2.9065744e-03 3.9215689e-03 ... 3.9215689e-03
   1.2302962e-04 3.6447521e-03]
  [2.9680892e-03 3.9215689e-03 9.2272203e-05 ... 7.6893506e-05
   3.6601308e-03 3.0296040e-03]
  ...
  [3.0296040e-03 3.2910421e-03 3.9215689e-03 ... 3.9215689e-03
   0.0000000e+00 3.1218762e-03]
  [3.2602844e-03 3.9215689e-03 0

In [33]:
# savemodel
# testAttrX
# testImagesX

alexnetPredictTest = np.array(alexnet.predict(test_images))
resnetPredictTest = np.array(resnet.predict(test_images))
mobilenetPredictTest = np.array(mobilenet.predict(test_images))
print(alexnetPredictTest.shape)
print(resnetPredictTest.shape)
print(mobilenetPredictTest.shape)
print(gasdata.shape)



6/6 [==============================] - 3s 431ms/step
(176, 1)
(176, 1)
(176, 1)
(176, 2)


In [34]:
testData = concatenate([alexnetPredictTest, resnetPredictTest, mobilenetPredictTest, gasdata])
print(testData)
print(testData.shape)

tf.Tensor(
[[ 0.0000000e+00  2.0797552e-04 -3.2976783e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  2.1014891e-04 -3.2905097e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  2.0960174e-04 -3.2890309e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  1.9809630e-04 -3.2880962e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  1.9272935e-04 -3.2704314e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  1.7790792e-04 -3.2430620e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  1.9180485e-04 -3.2244320e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  1.9246970e-04 -3.2589949e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  1.7759670e-04 -3.2349992e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  1.9746204e-04 -3.2276014e-21  1.0000000e+00
   1.0000000e+00]
 [ 0.0000000e+00  1.9925299e-04 -3.1830926e-21  2.7408257e-01
   2.7866971e-01]
 [ 0.0000000e+00  1.9175580e-04 -3.1483665e-21  2.7293578e-01
   2.7866971e-01]
 [ 0.0000000e+00  1.9351272e-

In [35]:
print(alexnetPredict)
# print(resnetPredict)
# print(mobilenetPredict)
print("gay")
print(alexnetPredictTest)
# print(resnetPredictTest)
# print(mobilenetPredictTest)

[[0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+0

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
performance = model.predict(testData)
print(performance)
actual = []
for value in performance: 
    #print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)
acutal = np.array(actual)
print(result)
accuracy = accuracy_score(result, actual)
print('Accuracy: %f' % accuracy)
precision = precision_score(result, actual)
print('Precision: %f' % precision)
recall = recall_score(result, actual)
print('Recall: %f' % recall)
f1 = f1_score(result, actual)
print('F1 score: %f' % f1)

[2023-12-28 01:08:35.824] Start to evalute the model:
[2023-12-28 01:08:35.824] Evaluating cascade layer = 0 


[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
0      0
1      0
2      0
3      0
4      0
      ..
171    1
172    1
173    1
174    1
175    1
Name: Gas, Length: 176, dtype: int64
Accuracy: 0.880682
Precision: 0.843750
Recall: 0.830769
F1 score: 0.837209
